Much of the code here is based on the [tutorial on the edward homepage](http://edwardlib.org/getting-started).

In [12]:
%matplotlib inline
#from __future__ import absolute_import
#from __future__ import division
#from __future__ import print_function

import edward as ed
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

import statsmodels.api as sm
import pandas as pd
plt.style.use('ggplot')

### Generate Toy Data

In [13]:
#super basic example: linear regression with homoskedastic errors
#Y~N(mu+Xb,sigma2)
# X is an 50x9 matrix where each column is gaussian(5,sd=3)
N = 50
Ntest = 40
Xtrain = np.array([np.random.normal(5,3,size=N) for i in range(9)]).T
Xtest = np.array([np.random.normal(5,3,size=Ntest) for i in range(9)]).T
#X is a 50x9 design matrix of continuous covariates
#print("colmeans: ", np.mean(X,0)) #column means, should be about 5
#print("col sds: ", np.std(X,0)) #column stdevs, should be about 3
beta0 = 33
beta = np.array([-3,-1.5,-.5,-.25,-0.01,0.01,.5,1,1.5])
#strong negative: 1,2
#weak negative: 3,4
#insignificant effects: 5,6
#weak positive: 7
#strong positive: 8,9
sigma = 4 #fairly high level of noise
yhat = Xtrain.dot(beta)+beta0
ytrain = yhat+np.random.normal(0,sigma,size=N)
ytest = Xtest.dot(beta)+beta0+np.random.normal(0,sigma,size=Ntest)
df = pd.DataFrame(np.hstack((ytrain.reshape(N,1),Xtrain)))
df.columns = ["y"]+["x"+str(i) for i in range(1,10)]
dftest = pd.DataFrame(np.hstack((ytest.reshape(Ntest,1),Xtest)))
dftest.columns = df.columns
df["intercept"]=1.0
dftest["intercept"] = 1.0
#df.head()
df.describe()

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,intercept
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.0
mean,24.038840,4.413367,5.134101,4.317427,5.095714,4.368622,5.568713,5.822354,5.006309,4.672567,1.0
std,10.687380,2.618149,2.883398,2.803299,3.212930,3.175544,2.876165,2.989535,3.340791,3.293853,0.0
min,1.390038,-1.160841,-0.336608,-2.503885,-3.023784,-1.811805,0.027005,-1.374110,-3.734738,-2.741706,1.0
25%,17.818256,2.318125,2.573311,2.321118,2.708612,1.961760,4.007009,4.050411,3.081040,2.258755,1.0
50%,23.547641,4.682267,5.511717,4.562429,4.896201,4.272829,5.692052,5.680239,4.894908,4.541672,1.0
75%,31.712101,6.350367,7.163294,5.997934,7.417331,6.386332,7.569856,7.870201,6.686562,6.551063,1.0
max,51.300863,10.559016,11.215865,10.167758,11.526455,10.031915,13.179536,11.988916,14.396580,13.319129,1.0


### Maximum Likelihood

First we will try the frequentist approach of maximum likelihood, provided in python by the **statsmodels** package

In [14]:
mod1 = sm.GLM(df["y"],df[df.columns[1:]],family=sm.families.Gaussian())
mod1 = mod1.fit()
mod1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                   50
Model:                            GLM   Df Residuals:                       40
Model Family:                Gaussian   Df Model:                            9
Link Function:               identity   Scale:                   13.7198829887
Method:                          IRLS   Log-Likelihood:                -130.84
Date:                Wed, 22 Mar 2017   Deviance:                       548.80
Time:                        14:48:59   Pearson chi2:                     549.
No. Iterations:                     4                                         
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
x1            -2.9526      0.216    -13.684      0.000        -3.376    -2.530
x2            -1.3272      0.219     -6.058      0.000        -1.757    -0.898
x3            -0.3150      0.205     -1.536      0.125        -0.717     0.087
x4            -0.1249      0.203     -0.614      0.539        -0.523     0.274
x5             0.0123      0.180      0.069      0.945        -0.340     0.364
x6             0.1679      0.210      0.798      0.425        -0.244     0.580
x7             0.9011      0.228      3.956      0.000         0.455     1.348
x8             1.0703      0.174      6.139      0.000         0.729     1.412
x9             1.5869      0.186      8.538      0.000         1.223     1.951
intercept     26.8718      3.349      8.024      0.000        20.308    33.436
==============================================================================
"""

In [15]:
ypred1 = mod1.predict(dftest[dftest.columns[1:]])
#root mean square prediction error
err1 = np.mean((ypred1-ytest)**2)
err1 #OLS prediction error

13.5217933856571

## Edward

### Maximum Likelihood

Edward supports maximum likelihood estimation (point estimates) of model parameters. However, you cannot actually inspect the parameter values. They are hidden inside the result object.

In [27]:
from edward.models import Normal

D = Xtest.shape[1]
X = tf.placeholder(tf.float32, [N, D]) #placeholder for data
#define likelihood and specify model parameters as variables (ie no priors)
b0 = tf.Variable(0.0)
#b0 = tf.Print(b0,[b0])
#b = tf.Variable(tf.zeros(D))
b = Normal(mu=tf.zeros(D), sigma=100*tf.ones(D))
y = Normal(mu=ed.dot(X, b) + b0, sigma=sigma*tf.ones(N))
#use empty dict {} to force MLE instead of bayesian
mle = ed.MAP([b], {y:ytrain,X:Xtrain})
mle.run()

Iteration    1 [  0%]: Loss = 753.438
Iteration  100 [ 10%]: Loss = 206.568
Iteration  200 [ 20%]: Loss = 202.564
Iteration  300 [ 30%]: Loss = 198.761
Iteration  400 [ 40%]: Loss = 195.450
Iteration  500 [ 50%]: Loss = 192.716
Iteration  600 [ 60%]: Loss = 190.522
Iteration  700 [ 70%]: Loss = 188.789
Iteration  800 [ 80%]: Loss = 187.432
Iteration  900 [ 90%]: Loss = 186.372
Iteration 1000 [100%]: Loss = 185.545


In [28]:
#inspect the estimated values for the parameters
sess = ed.get_session()
sess.run(b0)

17.429686

In [29]:
b.eval()

array([-111.49196625, -123.86114502, -161.34527588,  -39.96683121,
       -170.55706787, -113.93430328,  152.57043457,  -18.46553421,
        123.71065521], dtype=float32)

In [30]:
#sess = ed.get_session()
#form posterior predictive distribution
Xt = tf.placeholder(tf.float32, [Ntest,D])
yt = Normal(ed.dot(Xt,b)+b0,sigma=sigma*tf.ones(Ntest))
print("Mean squared error on test data:")
print(ed.evaluate('mean_squared_error', data={Xt: Xtest, yt: ytest}))

Mean squared error on test data:
4287.95


### Maximum A Posteriori
Now we will try to solve the same problem using Edward. First attempt is MAP estimation, some code here is copied from [Edward tutorial](http://edwardlib.org/tutorials/supervised-regression)

In [31]:
#define priors for model parameters
b = Normal(mu=tf.zeros(D), sigma=tf.ones(D))
b0 = Normal(mu=tf.zeros(1), sigma=tf.ones(1))
inference = ed.MAP([b0,b], {y:ytrain,X:Xtrain})
inference.run()

Iteration    1 [  0%]: Loss = 2977096.500
Iteration  100 [ 10%]: Loss = 1334589.125
Iteration  200 [ 20%]: Loss = 1932861.125
Iteration  300 [ 30%]: Loss = 1370627.250
Iteration  400 [ 40%]: Loss = 1168183.125
Iteration  500 [ 50%]: Loss = 8019773.000
Iteration  600 [ 60%]: Loss = 7719394.500
Iteration  700 [ 70%]: Loss = 1694345.250
Iteration  800 [ 80%]: Loss = 1142396.875
Iteration  900 [ 90%]: Loss = 9116440.000
Iteration 1000 [100%]: Loss = 3469600.250


In [35]:
b0.eval()

array([-0.56545317], dtype=float32)

In [33]:
yt = Normal(ed.dot(Xt,b)+b0,sigma=sigma*tf.ones(Ntest))
print("Mean squared error on test data:")
print(ed.evaluate('mean_squared_error', data={Xt: Xtest, yt: ytest}))

Mean squared error on test data:
791.231


Iteration    1 [  0%]: Loss = 3020.843
Iteration  100 [ 10%]: Loss = 244.376
Iteration  200 [ 20%]: Loss = 220.132
Iteration  300 [ 30%]: Loss = 219.095
Iteration  400 [ 40%]: Loss = 218.850
Iteration  500 [ 50%]: Loss = 218.782
Iteration  600 [ 60%]: Loss = 218.765
Iteration  700 [ 70%]: Loss = 218.761
Iteration  800 [ 80%]: Loss = 218.760
Iteration  900 [ 90%]: Loss = 218.759
Iteration 1000 [100%]: Loss = 218.759


In [149]:
dir(inference)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'build_loss_and_gradients',
 'coord',
 'data',
 'debug',
 'finalize',
 'increment_t',
 'initialize',
 'latent_vars',
 'logging',
 'loss',
 'n_iter',
 'n_print',
 'print_progress',
 'run',
 'scale',
 't',
 'threads',
 'train',
 'update']

In [141]:
ytrain.shape

(50, 1)